<img src="https://raw.githubusercontent.com/rhatiro/Curso_EBAC-Profissao_Cientista_de_Dados/main/ebac-course-utils/media/logo/newebac_logo_black_half.png" alt="ebac-logo">

---

# **Profissão: Cientista de Dados**
### **Módulo 13** | Exercício 2 | Regressão Múltipla

Aluno [Lays Félix](https://www.linkedin.com/in/laysfelixbusiness)<br>
Data: 02 de julho de 2024

---

#### Previsão de renda II

Vamos continuar trabalhando com a base 'previsao_de_renda.csv', que é a base do seu próximo projeto. Vamos usar os recursos que vimos até aqui nesta base.

|variavel|descrição|
|-|-|
|data_ref                | Data de referência de coleta das variáveis |
|index                   | Código de identificação do cliente|
|sexo                    | Sexo do cliente|
|posse_de_veiculo        | Indica se o cliente possui veículo|
|posse_de_imovel         | Indica se o cliente possui imóvel|
|qtd_filhos              | Quantidade de filhos do cliente|
|tipo_renda              | Tipo de renda do cliente|
|educacao                | Grau de instrução do cliente|
|estado_civil            | Estado civil do cliente|
|tipo_residencia         | Tipo de residência do cliente (própria, alugada etc)|
|idade                   | Idade do cliente|
|tempo_emprego           | Tempo no emprego atual|
|qt_pessoas_residencia   | Quantidade de pessoas que moram na residência|
|renda                   | Renda em reais|

In [1]:
import numpy as np
import pandas as pd
import patsy

import matplotlib.pyplot as plt
import seaborn as sns

import patsy
import statsmodels.api as sm
import statsmodels.formula.api as smf

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import r2_score

In [2]:
prev_renda = pd.read_csv('previsao_de_renda.csv')

In [3]:
prev_renda.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             15000 non-null  int64  
 1   data_ref               15000 non-null  object 
 2   id_cliente             15000 non-null  int64  
 3   sexo                   15000 non-null  object 
 4   posse_de_veiculo       15000 non-null  bool   
 5   posse_de_imovel        15000 non-null  bool   
 6   qtd_filhos             15000 non-null  int64  
 7   tipo_renda             15000 non-null  object 
 8   educacao               15000 non-null  object 
 9   estado_civil           15000 non-null  object 
 10  tipo_residencia        15000 non-null  object 
 11  idade                  15000 non-null  int64  
 12  tempo_emprego          12427 non-null  float64
 13  qt_pessoas_residencia  15000 non-null  float64
 14  renda                  15000 non-null  float64
dtypes:

1. Separe a base em treinamento e teste (25% para teste, 75% para treinamento).
2. Rode uma regularização *ridge* com alpha = [0, 0.001, 0.005, 0.01, 0.05, 0.1] e avalie o $R^2$ na base de testes. Qual o melhor modelo?
3. Faça o mesmo que no passo 2, com uma regressão *LASSO*. Qual método chega a um melhor resultado?
4. Rode um modelo *stepwise*. Avalie o $R^2$ na vase de testes. Qual o melhor resultado?
5. Compare os parâmetros e avalie eventuais diferenças. Qual modelo você acha o melhor de todos?
6. Partindo dos modelos que você ajustou, tente melhorar o $R^2$ na base de testes. Use a criatividade, veja se consegue inserir alguma transformação ou combinação de variáveis.
7. Ajuste uma árvore de regressão e veja se consegue um $R^2$ melhor com ela.

In [4]:
# exclui unnamed, id_cliente e data_ref
X = prev_renda.drop(['Unnamed: 0', 'data_ref', 'id_cliente'], axis=1)

In [5]:
X.head()

,sexo,posse_de_veiculo,posse_de_imovel,qtd_filhos,tipo_renda,educacao,estado_civil,tipo_residencia,idade,tempo_emprego,qt_pessoas_residencia,renda
0,F,False,True,0,Empresário,Secundário,Solteiro,Casa,26,6.602740,1.0,8060.34
1,M,True,True,0,Assalariado,Superior completo,Casado,Casa,28,7.183562,2.0,1852.15
2,F,True,True,0,Empresário,Superior completo,Casado,Casa,35,0.838356,2.0,2253.89
3,F,False,True,1,Servidor público,Superior completo,Casado,Casa,30,4.846575,3.0,6600.77
4,M,True,False,0,Assalariado,Secundário,Solteiro,Governamental,33,4.293151,1.0,6475.97


1. Separe a base em treinamento e teste (25% para teste, 75% para treinamento).

In [6]:
# Definindo o target
y = prev_renda['renda']

In [7]:
y.head()

0    8060.34
1    1852.15
2    2253.89
3    6600.77
4    6475.97
Name: renda, dtype: float64

In [8]:
# Definindo a proporção de teste e treinamento

prev_renda_train, prev_renda_test = train_test_split(prev_renda, test_size=0.25, random_state=100)

In [9]:
modelo = '''renda ~ sexo 
                    + posse_de_veiculo 
                    + posse_de_imovel 
                    + qtd_filhos 
                    + tipo_renda 
                    + educacao 
                    + estado_civil 
                    + tipo_residencia 
                    + idade 
                    + tempo_emprego 
                    + qt_pessoas_residencia
                    '''

y_train, X_train = patsy.dmatrices(formula_like=modelo, data=prev_renda_train)
y_test, X_test = patsy.dmatrices(formula_like=modelo, data=prev_renda_test)

2. Rode uma regularização ridge com alpha = [0, 0.001, 0.005, 0.01, 0.05, 0.1] e avalie o  𝑅2 na base de testes. Qual o melhor modelo?

In [17]:
alphas = [0, 0.001, 0.005, 0.01, 0.05, 0.1]
r2_y_pred = []

for alpha in alphas:
    md = sm.OLS(y_test, X_test)
    reg = md.fit_regularized(method='elastic_net', 
                             refit=True, 
                             L1_wt=0,  # ridge fit
                             alpha=alpha)
    y_pred = reg.predict(X_test)
    r2 = r2_score(y_test, y_pred)
    
    r2_y_pred.append(r2)
    print(f'Alpha {alpha}: \nR-squared = {r2}\n')
    
pd.DataFrame({'alpha':alphas, 
              '𝑅2':r2_y_pred
             }).sort_values(by='𝑅2', ascending=False)

Alpha 0: 
R-squared = 0.2994510060587443

Alpha 0.001: 
R-squared = 0.29941838203390936

Alpha 0.005: 
R-squared = 0.2991127065652508

Alpha 0.01: 
R-squared = 0.2987142447364981

Alpha 0.05: 
R-squared = 0.29436799479772535

Alpha 0.1: 
R-squared = 0.2870648054133397



,alpha,𝑅2
0,0.000,0.299451
1,0.001,0.299418
2,0.005,0.299113
3,0.010,0.298714
4,0.050,0.294368
5,0.100,0.287065


O que obteve o melhor modelo é o que teve valor de alpha igual a 0.000, resultando em um R² de 0.299451, pois teve o coeficiente de determinação maior entre os modelos de regressão testados com regularização RIDGE.

3. Faça o mesmo que no passo 2, com uma regressão LASSO. Qual método chega a um melhor resultado?

In [18]:
alphas = [0, 0.001, 0.005, 0.01, 0.05, 0.1]
r2_y_pred = []

for alpha in alphas:
    md = sm.OLS(y_test, X_test)
    reg = md.fit_regularized(method='elastic_net', 
                             refit=True, 
                             L1_wt=1,  # lasso fit
                             alpha=alpha)
    y_pred = reg.predict(X_test)
    r2 = r2_score(y_test, y_pred)
    
    r2_y_pred.append(r2)
    print(f'Alpha {alpha}: \nR-squared = {r2}\n')
    
pd.DataFrame({'alpha':alphas, 
              '𝑅2':r2_y_pred
             }).sort_values(by='𝑅2', ascending=False)

Alpha 0: 
R-squared = 0.2994510060587444

Alpha 0.001: 
R-squared = 0.2994510060587444

Alpha 0.005: 
R-squared = 0.2994510060587444

Alpha 0.01: 
R-squared = 0.2994510060587444

Alpha 0.05: 
R-squared = 0.2994510060587444

Alpha 0.1: 
R-squared = 0.2994510060587444



,alpha,𝑅2
0,0.000,0.299451
1,0.001,0.299451
2,0.005,0.299451
3,0.010,0.299451
4,0.050,0.299451
5,0.100,0.299451


Todos os modelos tiverem melhor resultado, resultando em um R² de 0.299451 entre os modelos de regressão testados com regularização LASSO.

4. Rode um modelo stepwise. Avalie o  𝑅2 na fase de testes. Qual o melhor resultado?

In [19]:
df_dummies = pd.get_dummies(data=prev_renda.dropna(), drop_first=True).astype(int)

X = df_dummies.drop(columns='renda')
y = df_dummies['renda']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [20]:
def stepwise_selection(X, y, 
                       initial_list=[], 
                       threshold_in=0.05, 
                       threshold_out = 0.05, 
                       verbose=True):
    """ Perform a forward-backward feature selection 
    based on p-value from statsmodels.api.OLS
    Arguments:
        X - pandas.DataFrame with candidate features
        y - list-like with the target
        initial_list - list of features to start with (column names of X)
        threshold_in - include a feature if its p-value < threshold_in
        threshold_out - exclude a feature if its p-value > threshold_out
        verbose - whether to print the sequence of inclusions and exclusions
    Returns: list of selected features 
    Always set threshold_in < threshold_out to avoid infinite looping.
    See https://en.wikipedia.org/wiki/Stepwise_regression for the details
    """
    included = list(initial_list)
    while True:
        changed=False
        
        # forward step
        excluded = list(set(X.columns)-set(included))
        new_pval = pd.Series(index=excluded, dtype=np.dtype('float64'))
        for new_column in excluded:
            model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included+[new_column]]))).fit()
            new_pval[new_column] = model.pvalues[new_column]
        best_pval = new_pval.min()
        if best_pval < threshold_in:
            best_feature = new_pval.index[new_pval.argmin()]
            included.append(best_feature)
            changed=True
            if verbose:
                 print('Add  {:30} with p-value {:.6}'.format(best_feature, best_pval))

        # backward step
        print("#############")
        print(included)
        model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included]))).fit()
        # use all coefs except intercept
        pvalues = model.pvalues.iloc[1:]
        worst_pval = pvalues.max() # null if pvalues is empty
        if worst_pval > threshold_out:
            changed=True
            worst_feature = pvalues.argmax()
            included.remove(worst_feature)
            if verbose:
                print('Drop {:30} with p-value {:.6}'.format(worst_feature, worst_pval))
        if not changed:
            break
    return included

variaveis = stepwise_selection(X_test, y_test)

print('\n\nresulting features:')
print(variaveis)

Add  tempo_emprego                  with p-value 1.15416e-120
#############
['tempo_emprego']
Add  sexo_M                         with p-value 1.97372e-116
#############
['tempo_emprego', 'sexo_M']
Add  educacao_Superior completo     with p-value 0.00030172
#############
['tempo_emprego', 'sexo_M', 'educacao_Superior completo']
Add  idade                          with p-value 0.000294673
#############
['tempo_emprego', 'sexo_M', 'educacao_Superior completo', 'idade']
Add  posse_de_veiculo               with p-value 0.00268263
#############
['tempo_emprego', 'sexo_M', 'educacao_Superior completo', 'idade', 'posse_de_veiculo']
Add  tipo_renda_Empresário          with p-value 0.0127891
#############
['tempo_emprego', 'sexo_M', 'educacao_Superior completo', 'idade', 'posse_de_veiculo', 'tipo_renda_Empresário']
Add  qt_pessoas_residencia          with p-value 0.0167073
#############
['tempo_emprego', 'sexo_M', 'educacao_Superior completo', 'idade', 'posse_de_veiculo', 'tipo_renda_Empresário

In [22]:
X = df_dummies[variaveis]
y = df_dummies['renda']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

y_pred = sm.OLS(y_test, X_test).fit().predict(X_test)
print(f'R-squared na base de testes é {r2_score(y_test, y_pred)}')

R-squared na fase de testes é 0.2994510060587444


In [23]:
X = df_dummies[variaveis]
y = df_dummies['renda']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.75, random_state=42)

y_pred = sm.OLS(y_train, X_train).fit().predict(X_train)
print(f'R-squared na base de treino é {r2_score(y_train, y_pred)}')

R-squared na base de treino é 0.25044109397253045


In [24]:
alphas = [0, 0.001, 0.005, 0.01, 0.05, 0.1]
r2_y_pred = []

for alpha in alphas:
    md = sm.OLS(y_train, X_train)
    reg = md.fit_regularized(method='elastic_net', 
                             refit=True, 
                             L1_wt=0,  # ridge fit
                             alpha=alpha)
    y_pred = reg.predict(X_test)
    r2 = r2_score(y_test, y_pred)
    
    r2_y_pred.append(r2)
    print(f'Alpha {alpha}: \nR-squared = {r2}\n')
    
pd.DataFrame({'alpha':alphas, 
              '𝑅2':r2_y_pred
             }).sort_values(by='𝑅2', ascending=False)

Alpha 0: 
R-squared = 0.24802340450427973

Alpha 0.001: 
R-squared = 0.24807451486522514

Alpha 0.005: 
R-squared = 0.2482132969030879

Alpha 0.01: 
R-squared = 0.24827541693169075

Alpha 0.05: 
R-squared = 0.24612772428323226

Alpha 0.1: 
R-squared = 0.24063395432504509



,alpha,𝑅2
3,0.010,0.248275
2,0.005,0.248213
1,0.001,0.248075
0,0.000,0.248023
4,0.050,0.246128
5,0.100,0.240634


In [25]:
alphas = [0, 0.001, 0.005, 0.01, 0.05, 0.1]
r2_y_pred = []

for alpha in alphas:
    md = sm.OLS(y_train, X_train)
    reg = md.fit_regularized(method='elastic_net', 
                             refit=True, 
                             L1_wt=1,  # lasso fit
                             alpha=alpha)
    y_pred = reg.predict(X_test)
    r2 = r2_score(y_test, y_pred)
    
    r2_y_pred.append(r2)
    print(f'Alpha {alpha}: \nR-squared = {r2}\n')
    
pd.DataFrame({'alpha':alphas, 
              '𝑅2':r2_y_pred
             }).sort_values(by='𝑅2', ascending=False)

Alpha 0: 
R-squared = 0.24802340450427962

Alpha 0.001: 
R-squared = 0.24802340450427962

Alpha 0.005: 
R-squared = 0.24802340450427962

Alpha 0.01: 
R-squared = 0.24802340450427962

Alpha 0.05: 
R-squared = 0.24802340450427962

Alpha 0.1: 
R-squared = 0.24802340450427962



,alpha,𝑅2
0,0.000,0.248023
1,0.001,0.248023
2,0.005,0.248023
3,0.010,0.248023
4,0.050,0.248023
5,0.100,0.248023


O melhor resultado entre os modelos de regressão testados na base de testes e treino foi o que apresentou o R² de 0.299451 com regularização LASSO e RIDGE.

5. Compare os parâmetros e avalie eventuais diferenças. Qual modelo você acha o melhor de todos?

A meu ver o melhor modelo de todos foi o modelo de regressão testado com regularização LASSO onde todos os alphas verificados tiveram o mesmo R² 0.299451.

6. Partindo dos modelos que você ajustou, tente melhorar o  𝑅2 na base de testes. Use a criatividade, veja se consegue inserir alguma transformação ou combinação de variáveis.


In [29]:
# usando as variáveis da função stepwise_selection ---> resulting features:

df_matrix = df_dummies.loc[:, variaveis + ['renda']]
df_matrix.rename(columns={'educacao_Superior completo': 'educacao_Superior_completo'}, 
                 inplace=True)

df_train, df_test = train_test_split(df_matrix, test_size=0.25, random_state=42)

modelo = smf.ols(formula='''
                         np.log(renda) ~ tempo_emprego 
                                         + sexo_M 
                                         + educacao_Superior_completo
                                         + idade 
                                         - posse_de_veiculo 
                                         + tipo_renda_Empresário 
                                         + qt_pessoas_residencia
                                         - tipo_residencia_Estúdio
                                         ''', data=df_test)

reg = modelo.fit_regularized(method='elastic_net', 
                             refit=True, 
                             L1_wt=1,
                             alpha=0)

print(reg.summary())

y_pred = reg.predict(df_test)
print('\nCoeficiente de determinação (𝑅2) na base de testes:', 
      r2_score(df_test['renda'], 
               np.exp(y_pred)
              )
     )

                            OLS Regression Results                            
Dep. Variable:          np.log(renda)   R-squared:                       0.363
Model:                            OLS   Adj. R-squared:                  0.362
Method:                 Least Squares   F-statistic:                     252.7
Date:                Tue, 02 Jul 2024   Prob (F-statistic):          3.05e-298
Time:                        10:12:58   Log-Likelihood:                -3388.8
No. Observations:                3107   AIC:                             6794.
Df Residuals:                    3100   BIC:                             6842.
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
Intercept           

7. Ajuste uma árvore de regressão e veja se consegue um  𝑅2 melhor com ela.

In [30]:
X = df_dummies[variaveis]
y = df_dummies['renda']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

reg_tree = DecisionTreeRegressor(random_state=42, max_depth=5, min_samples_leaf=13)
reg_tree.fit(X_train, y_train)

DecisionTreeRegressor(max_depth=5, min_samples_leaf=13, random_state=42)

In [31]:
print('Coeficiente de determinação (𝑅2) na base de testes:', 
      reg_tree.score(X=X_test, y=y_test))

Coeficiente de determinação (𝑅2) na base de testes: 0.389085096059829
